In [1]:
# import langchain.agents import load_tools
import os
import langchain
from langchain.callbacks import get_openai_callback
from langchain.tools import  Tool, DuckDuckGoSearchResults, YouTubeSearchTool
from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
os.environ["OPENAI_API_KEY"] = "<your api key>"
os.environ["SERPAPI_API_KEY"] = "<your serpapi key>"

In [3]:
search = DuckDuckGoSearchResults()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information, Also for link and source. You should give priority to using it."
)

youtubesearch = YouTubeSearchTool()
youtube_tool = Tool.from_function(
    func = youtubesearch.run,
    name="youtubesearch",
    description="useful when you need link of video from youtube."   
)


In [18]:

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [25]:
tools = [search_tool, youtube_tool]

In [26]:
llm_with_tools = llm.bind_tools(tools)

/home/vedant/Documents/datascience/summer vacation project/personal_assistant/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  _obj_type = "attribute"


In [27]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''As my assistant, I rely on you to provide detailed and accurate answers to my queries. 
            Utilize your language abilities and internet search tools to fetch relevant resources that enhance my understanding of the topic. 
            Please include links to these resources along with your response. Your assistance in synthesizing information is highly valued.''',
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [28]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [29]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
with get_openai_callback() as cb:
    list(agent_executor.stream({"input": "What are LLMs in AI?"}))
    print(cb)




> Entering new AgentExecutor chain...
LLMs in AI refer to Language Model-based Machine Learning models in the field of Artificial Intelligence. LLMs are a type of machine learning model that uses language models to process and understand natural language data. These models are trained on large amounts of text data and can generate human-like text, answer questions, and perform various language-related tasks.

LLMs are typically based on deep learning techniques, such as recurrent neural networks (RNNs) or transformer models. They learn to predict the next word or sequence of words in a given context, which allows them to generate coherent and contextually relevant text.

Some popular examples of LLMs in AI include OpenAI's GPT (Generative Pre-trained Transformer) models, such as GPT-3, which have demonstrated impressive language generation capabilities. These models have been used for tasks like text completion, language translation, chatbots, and even creative writing.

LLMs have a 